<a href="https://colab.research.google.com/github/Mjloturco/musicstreamnet/blob/main/Parallel_Input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple Parallel Input

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datetime import datetime 
import numpy as np 
import tensorflow as tf
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed, Input
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model 


from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Data Preprocessing

### Time Series

In [6]:
# df = pd.read_csv('/content/drive/My Drive/cs137/project/spotify_daily_song_major.csv')
df = pd.read_csv('/content/drive/My Drive/cs137/project/spotify_daily_charts.csv')

In [7]:
df = df.drop(columns=['track_name', 'artist', 'position'])
df.head()

timesteps = df['date'].unique()

In [ ]:
df_pivot = pd.pivot_table(df, values='streams', index=['date'], columns=['track_id'])
df_pivot.head()

In [9]:
# filter out songs without a lot of stream counts 
df_pivot = df_pivot.dropna(axis=1, thresh=800)

# fill NaN values with 0???
# df_pivot = df_pivot.fillna(0)
df_pivot = df_pivot.interpolate(method='linear', axis=1, limit=2)
df_pivot = df_pivot.fillna(0)

In [10]:
# pd.options.display.max_rows = None
# df_pivot.iloc[:, 1]

In [11]:
# get last 20% of songs
split = df['date'].unique()[-int(1598*0.2)]

df_train = df_pivot.loc[:split]
df_test = df_pivot.loc[split:]

In [12]:
# min-max scaling 

from sklearn.preprocessing import MinMaxScaler 

l = [i for i in df_pivot if i != 'date']

scaler = MinMaxScaler() 
scaled_train = scaler.fit_transform(df_train[l])
scaled_test = scaler.fit_transform(df_test[l])


In [13]:
def split_sequence(sequence, look_back, forecast_horizon):
  """
  splitting sequence
  """
  X, y = list(), list() 
  
  for i in range(len(sequence)):

    lag_end = i + look_back
    forecast_end = lag_end + forecast_horizon 

    if forecast_end > len(sequence):
      break 

    seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
    X.append(seq_x)
    y.append(seq_y)
  
  return np.array(X), np.array(y)


# splitting parameters 
LOOK_BACK = 12
FORECAST_RANGE = 1
n_features = len(l)

X_train, y_train = split_sequence(scaled_train, LOOK_BACK, FORECAST_RANGE)
X_test, y_test = split_sequence(scaled_test, LOOK_BACK, FORECAST_RANGE)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(1268, 12, 63) (1268, 1, 63) (307, 12, 63) (307, 1, 63)


### Metadata

In [ ]:
meta = pd.read_csv('/content/drive/My Drive/cs137/project/spotify_tracks_augmented.csv').set_index('track_id')

# select only track IDs that we've filtered above 
meta = meta.loc[df_pivot.columns]


# drop columns 
dropcols = ['Unnamed: 0', 'artist_id', 'type', 'id', 'uri', 
            'track_href', 'analysis_url']

meta = meta.drop(columns=dropcols)

metasize = meta.shape[1]

# show metadata
print(meta.shape)
meta

## Model

In [27]:
# configure callbacks 

checkpoint_filepath = 'path_to_checkpoint_filepath'

checkpoint_callback = ModelCheckpoint(
 filepath=checkpoint_filepath,
 save_weights_only=False,
 monitor='val_loss',
 mode='min',
 save_best_only=True)

early_stopping_callback = EarlyStopping(
 monitor='val_loss',
 min_delta=0.005,
 patience=10,
 mode='min'
)

rlrop_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, mode='min', patience=3, min_lr=0.001)

### Sequential

In [28]:
EPOCHS = 100
BATCH_SIZE = 32
VALIDATION = 0.1 

# SEQUENTIAL MODEL 
seq_model = Sequential()
seq_model.add(LSTM(12, activation='relu', input_shape=(LOOK_BACK, n_features)))
seq_model.add(RepeatVector(FORECAST_RANGE))
seq_model.add(LSTM(12, activation='relu', return_sequences=True)) 
seq_model.add(TimeDistributed(Dense(n_features)))

seq_model.compile(optimizer='adam', loss='mse')

seq_history = seq_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, 
                    validation_split=VALIDATION, callbacks=[early_stopping_callback])

Epoch 1/100
36/36 [==============================] - 2s 15ms/step - loss: 0.1399 - val_loss: 0.0711
Epoch 2/100
36/36 [==============================] - 0s 6ms/step - loss: 0.1005 - val_loss: 0.0417
Epoch 3/100
36/36 [==============================] - 0s 6ms/step - loss: 0.0499 - val_loss: 0.0219
Epoch 4/100
36/36 [==============================] - 0s 6ms/step - loss: 0.0295 - val_loss: 0.0192
Epoch 5/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0241 - val_loss: 0.0192
Epoch 6/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0213 - val_loss: 0.0185
Epoch 7/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0192 - val_loss: 0.0178
Epoch 8/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0175 - val_loss: 0.0166
Epoch 9/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0157 - val_loss: 0.0156
Epoch 10/100
36/36 [==============================] - 0s 7ms/step - loss: 0.0139 - val_loss: 0.0145
Epoch 11

### Functional

In [29]:
swapped = np.swapaxes(meta, 0, 1)

swapped.shape

(13, 63)

In [107]:
y_train[:, 0, 1]

array([0.15871822, 0.12787845, 0.12587796, ..., 0.09610414, 0.10121977,
       0.13171639])

In [147]:

EPOCHS = 100
BATCH_SIZE = 32
VALIDATION = 0.1 

#                examples, timesteps, song
# input train shape (1265, 12, 63)
# train out         (1265, 4, 63) 
# test in           (304, 12, 63)
# test out          (304, 4, 63)

song_count = 63

# MODEL 
time_series_input = Input(shape=(12, song_count))
lstm_out_size = 12
lstm_out = LSTM(lstm_out_size)(time_series_input)
repeat_out = RepeatVector(FORECAST_RANGE)(lstm_out)
lstm2_out = LSTM(12, activation='relu', return_sequences=True)(repeat_out) 
dense_layer = Dense(song_count)
td_out = TimeDistributed(dense_layer)(lstm2_out)

# METADATA + concatenation
meta_input = Input(shape=(metasize, song_count))
full_features = concatenate([td_out, meta_input], axis=1)
swapped_features = tf.keras.layers.Permute((2, 1))(full_features)
d1 = Dense(16, activation='relu')(swapped_features) 
d2 = Dense(8, activation='tanh')(d1)
d3 = Dense(4, activation='tanh')(d2)
d4 = Dense(1, activation='linear')(d3)
# final = tf.keras.layers.Permute((2, 1))(d3)

# #outputshape
# output = Dense(1, activation='linear')(d1)

# # create model 
fun_model = Model(inputs=[time_series_input, meta_input], outputs=d4)

# # compile 
# model.compile(loss='mse', optimizer='adam')
fun_model.compile(optimizer='adam', loss='mse')

# stack metadata to match shape 
tiled = np.tile(np.swapaxes(np.expand_dims(meta, axis=0), 1, 2), [1268, 1, 1])

print(fun_model.summary())

X = [X_train[:, :, :song_count], tiled[:, :, :song_count]]
# Y = y_train[:, :, :song_count]
Y = np.swapaxes(y_train[:, :, :song_count], 1, 2)
fun_history = fun_model.fit(x=X, y=Y, 
                            epochs=EPOCHS, batch_size=BATCH_SIZE,
                            validation_split=VALIDATION, 
                            callbacks=[early_stopping_callback])

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_47 (InputLayer)          [(None, 12, 63)]     0           []                               
                                                                                                  
 lstm_50 (LSTM)                 (None, 12)           3648        ['input_47[0][0]']               
                                                                                                  
 repeat_vector_25 (RepeatVector  (None, 1, 12)       0           ['lstm_50[0][0]']                
 )                                                                                                
                                                                                                  
 lstm_51 (LSTM)                 (None, 1, 12)        1200        ['repeat_vector_25[0][0]']

In [148]:
print(yhat.shape)
print(X_train.shape, y_train.shape)

(307, 63, 1)
(1268, 12, 63) (1268, 1, 63)


## Evaluation

In [149]:
# pick which model we're evaluating 

model = fun_model
history = fun_history 

print(X_test.shape, meta.shape)
testmeta = np.tile(np.swapaxes(np.expand_dims(meta, axis=0), 1, 2), [307, 1, 1])
print(testmeta.shape)

test = [X_test, testmeta]

(307, 12, 63) (63, 13)
(307, 13, 63)


In [143]:
# Helper functions for model eval


def inverse_transform(y_test, yhat):
  """
  revert data back to original scale for evaluation
  """
  y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])
  yhat_reshaped = yhat.reshape(-1, yhat.shape[-1])

  yhat_inverse = scaler.inverse_transform(yhat_reshaped)
  y_test_inverse = scaler.inverse_transform(y_test_reshaped)

  return yhat_inverse, y_test_inverse 

  
def evaluate_forecast(y_test_inverse, yhat_inverse):
  """
  Evaluation metrics 
  """
  mse_ = tf.keras.losses.MeanSquaredError()
  rmse_ = tf.keras.metrics.RootMeanSquaredError()
  mae_ = tf.keras.losses.MeanAbsoluteError()
  mape_ = tf.keras.losses.MeanAbsolutePercentageError() 

  mse = mse_(y_test_inverse,yhat_inverse)
  tf.print('mse:', mse)

  rmse = rmse_(y_test_inverse, yhat_inverse)
  tf.print('rmse:', rmse)

  mae = mae_(y_test_inverse,yhat_inverse)
  tf.print('mae:', mae)

  mape = mape_(y_test_inverse,yhat_inverse)
  tf.print('mape:', mape)



In [150]:
yhat = model.predict(test, verbose=0)

# yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)

print(y_test_inverse.shape, yhat_inverse.shape, yhat.shape)
# evaluate_forecast(y_test_inverse, yhat_inverse)

(307, 63) (307, 63) (307, 63, 1)


In [ ]:
yhat[:, 1, 0]

In [93]:
print(yhat_inverse.shape, y_test_inverse.shape, y_test.shape, yhat.shape)

(307, 63) (307, 63) (307, 1, 63) (307, 63, 1)


In [73]:
# timesteps, batches?, songs 
print(yhat.shape)
print(yhat_inverse.shape)

(307, 1, 63)
(307, 63)


In [ ]:
import matplotlib.pyplot as plt 

# visualize loss
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()


In [ ]:
# plt.plot(timesteps[-304:], yhat, label='pred')
# plt.plot(timesteps[-304:], y_test, label='test') 
# plt.show()
plt.figure(figsize=(32, 32))

ntimesteps = yhat.shape[0]

fig, axs = plt.subplots(8, 8, figsize=(32, 32))
for i, ax in enumerate(axs.flat):
  ax.plot(timesteps[-ntimesteps:], yhat[:, 0, i-1], label='pred')
  ax.plot(timesteps[-ntimesteps:], y_test[:, 0, i-1], label='test')

plt.legend()
plt.show()

# plt.plot(timesteps[-304:], yhat[:, 0, 16], label='pred')
# plt.plot(timesteps[-304:], y_test[:, 0, 16], label='test')
# plt.legend()
# plt.show()

